In [17]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


In [18]:
def prep_data():
    '''define the data for the problem, namely : distance matrix, number of vehicles in the VRP, and the index of the depot'''
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [19]:
# DEFINING THE DISTANCE CALLBACK
def distance(from_index, to_index):
    '''returns distance between the two indices : from and to index'''
    #convert routing variable indices to distance matrix nodes
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [32]:
def display_solution(data, manager, routing, solution):
    def get_routes(solution, routing, manager):
        routes = []
        max_route_distance = 0
        for route_num in range(data['num_vehicles']):
            index = routing.Start(route_num)
            route = [manager.IndexToNode(index)]
            route_distance = 0
            while not routing.IsEnd(index):
                prev_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(prev_index, index, route_num)
                route.append(manager.IndexToNode(index))
            routes.append((route, route_distance))
            max_route_distance = max(max_route_distance, route_distance)
        
        return routes, max_route_distance
    print('objective value : ')
    print(solution.ObjectiveValue())

    routes, max_route_distance = get_routes(solution, routing, manager)
    for i, route_tuple in enumerate(routes):
        print('Route {} : {}'.format(i, route_tuple[0]))
        print('Distance of the route: {}m'.format(route_tuple[1]))
    print('Maximum of the route distances: {}m'.format(max_route_distance))

    # max_route_distance = 0
    # for vehicle in range(len(data['num_vehicles'])):
    #     index = routing.Start(vehicle)
    #     plan_output = 'Route for vehicle {} :\n'.format(vehicle) 
    #     route_distance = 0
    #     while not routing.IsEnd(index)

        

In [33]:
def main():
    # Instantiate the data problem.
    data = prep_data()

    #CREATING THE INDEX MANAGER AND ROUTING OBJECT
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    routing.AddDimension(transit_callback_index, 0, 3000, True, 'Distance')
    distance_dimension = routing.GetDimensionOrDie('Distance')
    distance_dimension.SetGlobalSpanCostCoefficient(1000)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # SOLVE
    solution = routing.SolveWithParameters(search_parameters)

    # display results
    if solution:
        display_solution(data, manager, routing, solution)
    else:
        print('No feasible solution')

if __name__ == '__main__':
    main()

objective value : 
1558208
Route 0 : [0, 8, 6, 2, 5, 0]
Distance of the route: 1552m
Route 1 : [0, 7, 1, 4, 3, 0]
Distance of the route: 1552m
Route 2 : [0, 9, 10, 16, 14, 0]
Distance of the route: 1552m
Route 3 : [0, 12, 11, 15, 13, 0]
Distance of the route: 1552m
Maximum of the route distances: 1552m
